In [10]:
import sys
from PyQt5 import QtWidgets, QtCore, QtGui
from PyQt5.QtWidgets import QSystemTrayIcon, QApplication, QLabel, QMainWindow, QPushButton
from PyQt5.QtGui import QIcon
from PyQt5.QtCore import Qt
import tkinter as tk
import PIL.Image
import numpy as np
import cv2
import io
import json
import ctypes
from io import BytesIO
import win32clipboard
import glob
import os
from matplotlib import pyplot as plt
from IPython.display import clear_output, Image, display
from google.cloud.vision import types
from google.cloud import vision
import mss
import pyperclip
import mss.tools
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/poorv/Downloads/ocr_scripts/keys/direct-outlook-270501-c05c2d97d1c6.json"

In [21]:
def send_text_clipboard(text):
    pyperclip.copy(text)
    spam = pyperclip.paste()
    
def dtlh(path, lh):
    """Detects text in the file."""
    client = vision.ImageAnnotatorClient()
    if type(path) == str:
        with io.open(path, 'rb') as image_file:
            content = image_file.read()
            image = vision.types.Image(content=content)
    else:
        image = types.Image(content=cv2.imencode('.jpg', path)[1].tostring())
        
    response = client.text_detection(
    image=image,
    image_context={"language_hints": lh},
    )
    texts = response.text_annotations
    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))
    try:
        return texts[0].description
    except IndexError:
        print('no text in image')

In [61]:
class ocrWidget(QtWidgets.QWidget):
#     clicked = QtCore.pyqtSignal(object)
    
    def __init__(self, parent = None):
        super().__init__()
        root = tk.Tk()
        screen_width = root.winfo_screenwidth()
        screen_height = root.winfo_screenheight()
        self.setGeometry(0, 0, screen_width, screen_height)
        self.setWindowTitle(' ')
        self.begin = QtCore.QPoint()
        self.end = QtCore.QPoint()
        self.parent = parent
        self.setWindowOpacity(0.3)
        QtWidgets.QApplication.setOverrideCursor(
            QtGui.QCursor(QtCore.Qt.CrossCursor)
        )
        self.setWindowFlags(QtCore.Qt.FramelessWindowHint)
        QtWidgets.QShortcut(
            QtGui.QKeySequence("Escape"), self, activated=self.on_Escape
        )

    def paintEvent(self, event):
        qp = QtGui.QPainter(self)
        qp.setPen(QtGui.QPen(QtGui.QColor('black'), 3))
        qp.setBrush(QtGui.QColor(128, 128, 255, 128))
        qp.drawRect(QtCore.QRect(self.begin, self.end))

    def mousePressEvent(self, event):
        self.begin = event.pos()
        self.end = self.begin
        self.update()
    
    @QtCore.pyqtSlot()
    def on_Escape(self):
        print("main esp exit")
#         self.close()
        self.closeAndReturn()
        

    def mouseMoveEvent(self, event):
        self.end = event.pos()
        self.update()
    
    def closeAndReturn(self):
        self.close()
        self.parent.setWindowOpacity(1.)
        return

    def mouseReleaseEvent(self, event):
        previous = self.parent
        self.close()

        x1 = min(self.begin.x(), self.end.x())
        y1 = min(self.begin.y(), self.end.y())
        x2 = max(self.begin.x(), self.end.x())
        y2 = max(self.begin.y(), self.end.y())
        
        with mss.mss() as sct:
            # The screen part to capture
            if (y1 == y2 and x1 == x2):
                print("no region selected")
#                 return 
                self.closeAndReturn()
            
            monitor = {"top": y1, "left":x1, "width": abs(x2 - x1) , "height": abs(y2 - y1)}
            output = "sct-{top}x{left}_{width}x{height}.png".format(**monitor)
            sct_img = sct.grab(monitor)
            try:
                img = PIL.Image.frombytes("RGB", sct_img.size, sct_img.bgra, "raw", "BGRX")
                img = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2RGB)
                im_gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
                ret,thresh = cv2.threshold(im_gray,127,255,cv2.THRESH_BINARY_INV)
                ocr = dtlh(thresh, ["hi"])
                if ocr:
                    send_text_clipboard(ocr)
                    print(ocr)
                self.parent.setWindowOpacity(1.)
            except ValueError:
                print("invalid region selection. try again")
                self.closeAndReturn()

In [62]:
app = QtWidgets.QApplication(sys.argv)
window = ocrWidget()
window.show()
app.aboutToQuit.connect(app.deleteLater)
sys.exit(app.exec_())

main esp exit


AttributeError: 'NoneType' object has no attribute 'setWindowOpacity'

SystemExit: 0

In [ ]:
"""what do we want.
1. when we start the program, icon appears in sys tray
click sys tray, and gray screen appears to capture 
close the OCR widget (but not the main sys tray, when done)
click the sys tray again to start OCR again 

In [15]:
dtlh(r"C:\Users\poorv\Downloads\ocr_scripts\demo\pg191_excerpt.png", ["kn"])

"wego\n?weg 3 erata\n? ಎಬಿವು' eravu\nn. [move) pulling (C. (Kitt.)) [Ka.\nD504(a)]\nn. (move] 1 pulling, dragging\n2 spasm, cramp (My. (Kitt.)) [Ka. D504(a)].\n•ಎಬಿವು' eravu\n283) vt. (move] 1 to rub;\nto rub off or out; to smear gently 2 to stroke gently\n[Ka. D505].\n"

In [53]:
import sys
from PyQt5.QtWidgets import (QApplication, QMainWindow, QWidget, 
                             QVBoxLayout, QTextEdit, QPushButton)

from PyQt5.QtWidgets import QApplication, QDialog
from PyQt5.QtCore import QCoreApplication

In [ ]:
from PyQt5 import QtGui, QtCore
import sys


class First(QMainWindow):
    def __init__(self, parent=None):
        super(First, self).__init__(parent)
        self.pushButton = QPushButton("click me")

        self.setCentralWidget(self.pushButton)

        self.pushButton.clicked.connect(self.on_pushButton_clicked)
        self.dialogs = list()

    def on_pushButton_clicked(self):
        dialog = ocrWidget(self)
        self.dialogs.append(dialog)
        dialog.show()
        self.setWindowOpacity(0.)

def main():
    app = QtCore.QCoreApplication.instance()
    if app is None:
        app = QApplication(sys.argv)
    main = First()
    main.show()
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()

no text in image
no region selected
invalid region selection. try again
city()
lication.instance()
(sys.argv)
-

